In [ ]:
!pip install pmdarima

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objs as go 
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import BoxCoxEndogTransformer
import pmdarima as pm
from statsmodels.tsa.stattools import adfuller

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
calender= pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
calender.head(3)


In [ ]:
sales = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
sales.head(3)

In [ ]:
sales_dept = sales.groupby('dept_id').sum()
sales_dept.head(3)

In [ ]:
sales_item = sales.groupby('item_id').sum()
sales_item.head(3)

In [ ]:

sales_cat = sales.groupby('cat_id').sum().T
sales_cat.head(3)

In [ ]:
sales_cat['days'] =  sales_cat.index

# we got all of the Calender variables from columns
sales_cat.days

In [ ]:
sales_store = sales.groupby('store_id').sum()
sales_state = sales.groupby('state_id').sum()

In [ ]:
sales_calender = calender.iloc[:,[0,2,3,4,5,6,7]]

sales_cat = pd.merge(sales_calender,sales_cat,how = 'inner', left_on = 'd', right_on = 'days')

sales_cat['date'] = pd.to_datetime(sales_cat['date'])
sales_cat = sales_cat.set_index('date')


In [ ]:
sales_cat = sales_cat.iloc[:,[-4,-3,-2]]
sales_cat.head(3)

In [ ]:
sales_cat.tail(3)

In [ ]:
fig = go.Figure(data = [
    go.Scatter(y=sales_cat['2011-01':'2016-04'].FOODS, x=sales_cat.index, name= 'Foods'), 
          go.Scatter(y=sales_cat['2011-01':'2016-04'].HOBBIES, x=sales_cat.index, name = 'Hobbies'),
          go.Scatter(y=sales_cat['2011-01':'2016-04'].HOUSEHOLD, x=sales_cat.index, name = 'Household')
])

fig.update_layout(title_text="Sales by Category")
fig.show()

Big part of Sales occur by Foods

In [ ]:
sns.heatmap(sales_cat[['FOODS','HOBBIES','HOUSEHOLD']].corr(), annot = True,  cbar=False)

In [ ]:
sales_cat_monthly = sales_cat.iloc[:,[0,1,2]].resample('M').sum()[2:-1] #mensal resampling
sales_cat_weekly = sales_cat.iloc[:,[0,1,2]].resample('W').sum()[8:-1] #weekly resampling
sales_cat_bimonthly = sales_cat.iloc[:,[0,1,2]].resample('2M').sum()[1:-1] #bimonthy resamply

In [ ]:
    lists = ['FOODS','HOBBIES','HOUSEHOLD']

    for i in lists:
        fig = go.Figure(data = [
            go.Scatter(y = sales_cat_monthly[i],x = sales_cat_monthly[i].index,name = i)
        ])
        fig.update_layout(title_text=i)
        fig.show()

In [ ]:
#Decomposition

In [ ]:
 lists = ['FOODS','HOBBIES','HOUSEHOLD']
print('**************** Monthly *************')
for i in lists:
    
        sm.tsa.seasonal_decompose(sales_cat_monthly[i],period = 12).plot()


In [ ]:
 lists = ['FOODS','HOBBIES','HOUSEHOLD']
print('**************** Weekly *************')
for i in lists:
    
        sm.tsa.seasonal_decompose(sales_cat_weekly[i],period = 12).plot()


In [ ]:
# it was clearly observed that our series are Seasonal

In [ ]:
def acf_pacf(df,columns,lag):

    plot_acf(df[columns],lags = lag,title = columns)
    plot_pacf(df[columns],lags = lag, title = columns)
    
    
    plt.show()

In [ ]:
for i in lists:
    acf_pacf(sales_cat_monthly,i,12)

In [ ]:
for i in lists:
    acf_pacf(sales_cat_weekly,i,12)

In [ ]:
#it seems that i will continue on Monthly Series

In [ ]:

 lists = ['FOODS','HOBBIES','HOUSEHOLD']
    
for i in lists:
     
        print('P values for {} is = {}'.format(i,adfuller(sales_cat_monthly[i], autolag='AIC')[1]))  




In [ ]:
# The Food is Stationary but the others need to be implement

In [ ]:
sales_cat_monthly_food  = sales_cat_monthly.iloc[:,0]

In [ ]:
 sales_cat_monthly = sales_cat_monthly.iloc[:,1:3]

In [ ]:
deneme = sales_cat_monthly.diff().dropna()

In [ ]:
for i in ['HOBBIES','HOUSEHOLD']:
    print('P values for {} is = {}'.format(i,adfuller(deneme[i], autolag='AIC')[1]))

In [ ]:
#The food is stationary
sales_cat_monthly_hobbies = deneme.iloc[:,0] 

In [ ]:
sales_cat_monthly_household = sales_cat_monthly.iloc[:,1]
# we will get salesc_cat_monthly_household without transformation because with Log,diff() 
#and df.shift(1)/df methods could not find p < 0.05

In [ ]:
#Let start with Food
model_food = pm.auto_arima(sales_cat_monthly_food,
                     seasonal = True,m = 12,
                     d=1,D=1,
                     start_p = 1,start_q = 1,
                     end_p = 10, end_q =10,
                     trace = True,
                     error_action = 'ignore',
                     suppress_warnings=True)

model_food.summary()

In [ ]:
model_food = sm.tsa.statespace.SARIMAX(sales_cat_monthly_food,order = (2,1,0),seasonal_order = (1,1,1,12),trend = 'c')
results_food = model_food.fit()

In [ ]:
results_food.plot_diagnostics()
plt.show()


In [ ]:
food_predict = results_food.predict(start=50,end= 65)

In [ ]:
fig = go.Figure(data = [
    go.Scatter(y=sales_cat_monthly_food,x = sales_cat_monthly_food.index),
    go.Scatter(y=food_predict,x= food_predict.index)
])

fig.update_layout(title_text= 'Food')
fig.show()

In [ ]:
# We will continue for household but we will close difference
#because we transformed it to stationary with diff() method
model_hobbies = pm.auto_arima(sales_cat_monthly_hobbies,
                     seasonal = True,m = 12,
                     d=1,D=1,
                     start_p = 1,start_q = 1,
                     end_p = 10, end_q =10,
                     trace = True,
                     error_action = 'ignore',
                     suppress_warnings=True)

model_hobbies.summary()

In [ ]:
model_hobbies = sm.tsa.statespace.SARIMAX(sales_cat_monthly_hobbies,order = (2,1,1),seasonal_order = (0,1,1,12),trend = 'c')
results_hobbies = model_hobbies.fit()
results_hobbies.plot_diagnostics()
plt.show()


In [ ]:
hobbies_predict = results_hobbies.predict(start = 50, end = 65)
values_predict = np.cumsum(hobbies_predict) + sales_cat_monthly.iloc[-1,0]

In [ ]:

sales_cat_monthly_hobbies = sales_cat_monthly.iloc[:,0]

fig = go.Figure(data = [
    go.Scatter(y=sales_cat_monthly_hobbies,x = sales_cat_monthly_hobbies.index),
    go.Scatter(y=values_predict,x= values_predict.index)
])

fig.update_layout(title_text= 'Hobbies')
fig.show()

In [ ]:

model_household = pm.auto_arima(sales_cat_monthly_household,
                     seasonal = True,m = 12,
                     d=1,D=1,
                     start_p = 1,start_q = 1,
                     end_p = 10, end_q =10,
                     trace = True,
                     error_action = 'ignore',
                     suppress_warnings=True)

model_household.summary()

In [ ]:
model_households = sm.tsa.statespace.SARIMAX(sales_cat_monthly_household,order = (0,1,1),seasonal_order = (0,1,1,12),trend = 'c')
results_households = model_households.fit()
diff_forecast = results_households.predict(start = 50, end = 65)


In [ ]:
results_households.plot_diagnostics()
plt.show()

In [ ]:
 # let the plot final chart

fig = go.Figure(data = [
    go.Scatter(y=sales_cat_monthly_household,x = sales_cat_monthly_household.index),
    go.Scatter(y=diff_forecast,x= diff_forecast.index)
])

fig.update_layout(title_text= 'Household')
fig.show()